## Pip install dependencies

In [3]:
!pip install llama-index
!pip install llama-index-core==0.10.6.post1
!pip install llama-index-postprocessor-flag-embedding-reranker
!pip install git+https://github.com/FlagOpen/FlagEmbedding.git
!pip install llama-parse
%pip install -U llama-index llama-index-embeddings-nomic
%pip install llama-index-llms-ollama
!pip install redis


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
  Using cached llama_index_core-0.10.6.post1-py3-none-any.whl (15.4 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.18
    Uninstalling llama-index-core-0.11.18:
      Successfully uninstalled llama-index-core-0.11.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-parse 0.5.7 requires llama-index-core>=0.11.0, but you have llama-index-core 0.10.6.post1 which is incompatible.
llama-index 0.11.18 requires llama-index-core<0.12.0,>=0.11.18, but you have llama-index-core 0.10.6.post1 which is incompatible.
llama-index-readers-llama-parse 0.3.0 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.10.6.post1 which is incompatible.
llama-index-readers-file 0.2.2 requires llama-in

## Import Libraries

In [19]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.nomic import NomicEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

In [21]:
# Load the .env file
load_dotenv()

# Access the Nomic API key
nomic_api_key = os.getenv("NOMIC_API_KEY")

# Access to Llama-cloud API Key
llama_cloud_api_key = os.getenv("LLAMA_CLOUD_API_KEY")

# Access to OpenAI API Key
OpenAI_api_key = os.getenv("OPENAI_API_KEY")


#llm = Ollama(model="llama3.1:latest", request_timeout=120.0)

#embed_model = NomicEmbedding(
    #api_key=nomic_api_key,
    #dimensionality=128,
    #model_name="nomic-embed-text-v1.5",
#)

llm = OpenAI(api_key=OpenAI_api_key, model="gpt-3.5-turbo-0125")
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

In [6]:
from llama_parse import LlamaParse

documents = LlamaParse(result_type="markdown").load_data("/Users/angelmurillo/Desktop/OpenSource_RAG_LLM/data/AVD-005-MAB_INOCULUM_1_998pgs.pdf")

Started parsing the file under job_id 41af4460-2978-4bc4-a2e3-352e7abe8c64


In [7]:
documents[3].text

'# 1-6-3-1-6 (2)\n\nSC2 Media Warming Time\n\nN/A\nN/A\n360\n248\nN/A\n\n# 1-6-3-1-6 (3)\n\nSC3 Media Warming Time\n\nN/A\nN/A\n360\n258\nN/A\n\n# 1-9-10-8-10\n\n# (1) Transfer Viable Cell Concentration\n\n|1.20|N/A|5.80|4.99|x10^6|\n|---|---|---|---|---|\n|SC3 Vessel 1|SC3 Vessel 1|SC3 Vessel 1|SC3 Vessel 1|SC3 Vessel 1|\n\n# (2) Transfer Viable Cell Concentration\n\n|1.20|N/A|5.80|4.90|x10^6|\n|---|---|---|---|---|\n|SC3 Vessel 2|SC3 Vessel 2|SC3 Vessel 2|SC3 Vessel 2|SC3 Vessel 2|\n\n# (3) Transfer Viable Cell Concentration\n\n|1.20|N/A|5.80|5.47|x10^6|\n|---|---|---|---|---|\n|SC3 Vessel 3|SC3 Vessel 3|SC3 Vessel 3|SC3 Vessel 3|SC3 Vessel 3|\n\n# (4) Transfer Viable Cell Concentration\n\n|1.20|N/A|5.80|5.24|x10^6|\n|---|---|---|---|---|\n|SC3 Vessel 4|SC3 Vessel 4|SC3 Vessel 4|SC3 Vessel 4|SC3 Vessel 4|\n\n# (5) Transfer Viable Cell Concentration\n\n|1.20|N/A|5.80|4.42|x10^6|\n|---|---|---|---|---|\n|SC3 Vessel 5|SC3 Vessel 5|SC3 Vessel 5|SC3 Vessel 5|SC3 Vessel 5|\n\n# (6) Transfe

Comparing two different retrieval/query engine strategies:
Using raw Markdown text as nodes for building index and apply simple query engine for generating the results;
Using MarkdownElementNodeParser for parsing the LlamaParse output Markdown results and building recursive retriever query engine for generation.

In [8]:
from copy import deepcopy
from llama_index.core.schema import TextNode
from llama_index.core import VectorStoreIndex


def get_page_nodes(docs, separator="\n---\n"):
    """Split each document into page node, by separator."""
    nodes = []
    for doc in docs:
        doc_chunks = doc.text.split(separator)
        for doc_chunk in doc_chunks:
            node = TextNode(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata),
            )
            nodes.append(node)

    return nodes

In [9]:
page_nodes = get_page_nodes(documents)

In [10]:
len(page_nodes)

998

In [11]:
import time
from llama_index.core.node_parser import MarkdownElementNodeParser
from httpx import ReadTimeout

#node_parser = MarkdownElementNodeParser(
    #llm=Ollama(model="llama3.1:latest"), num_workers=8
#)

#def get_nodes_with_retries(documents, retries=3, delay=5): #had to incorporate retries and delays potentially due to API load. OpenAI doesnt have this issue.
    #for attempt in range(retries):
        #try:
            #nodes = node_parser.get_nodes_from_documents(documents)
            #return nodes
        #except ReadTimeout as e:
            #print(f"Attempt {attempt + 1} failed: {e}. Retrying in {delay} seconds...")
            #time.sleep(delay)
    #raise Exception("All retries failed. Unable to fetch nodes.")

# Run the function 
#nodes = get_nodes_with_retries(documents)

# node_parser function keeps failing with ollama due to API load. Will try with Open AI LLM

In [12]:
# node_parser using openAI gpt3.5, but will need to change llm configuration at the beginning. 

node_parser = MarkdownElementNodeParser(
    llm=OpenAI(model="gpt-3.5-turbo-0125"),
    num_workers=8
)

In [13]:
nodes = node_parser.get_nodes_from_documents(documents) 

1it [00:00, 9686.61it/s]
1it [00:00, 16644.06it/s]
3it [00:00, 43690.67it/s]
8it [00:00, 119837.26it/s]
3it [00:00, 39444.87it/s]
1it [00:00, 16384.00it/s]
3it [00:00, 40072.97it/s]
4it [00:00, 54120.05it/s]
0it [00:00, ?it/s]
5it [00:00, 67650.06it/s]
3it [00:00, 43539.49it/s]
1it [00:00, 12671.61it/s]
2it [00:00, 23366.60it/s]
4it [00:00, 48913.17it/s]
2it [00:00, 27060.03it/s]
3it [00:00, 41120.63it/s]
3it [00:00, 43996.20it/s]
3it [00:00, 42366.71it/s]
3it [00:00, 39945.75it/s]
2it [00:00, 29852.70it/s]
3it [00:00, 40072.97it/s]
4it [00:00, 54827.50it/s]
1it [00:00, 15827.56it/s]
3it [00:00, 40072.97it/s]
3it [00:00, 31936.32it/s]
3it [00:00, 48395.82it/s]
2it [00:00, 26630.50it/s]
3it [00:00, 35645.64it/s]
1it [00:00, 12483.05it/s]
1it [00:00, 17848.10it/s]
1it [00:00, 16710.37it/s]
1it [00:00, 12372.58it/s]
1it [00:00, 12228.29it/s]
1it [00:00, 14074.85it/s]
1it [00:00, 14074.85it/s]
1it [00:00, 11618.57it/s]
1it [00:00, 22795.13it/s]
1it [00:00, 13148.29it/s]
1it [00:00, 15141.8

In [14]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)
# base nodes are the text heavy sections (headers and paragraphs) and objects are the doc meta data

In [17]:
objects[2].get_content()

'This table provides information on the transfer viable cell concentration in SC3 Vessel 8, including the cell concentration values in cells/mL.,\nwith the following table title:\nTransfer Viable Cell Concentration in SC3 Vessel 8,\nwith the following columns:\n- Transfer Viable Cell Concentration: None\n- N/A: None\n- 4.50: None\n'

In [22]:
# Lets dump both indexed tables and page text into the vector index
recursive_index = VectorStoreIndex(nodes=base_nodes + objects + page_nodes)

In [23]:
# lets checkout an example content for a page_node
print(page_nodes[31].get_content())

# Trends of Critical / Other Process Parameters

# Table of Contents

|Instr. No.|Parameter|Min|Target|Max|Actual|UOM|
|---|---|---|---|---|---|---|
|1-5-27-11-6 (1)|BP120X1QI9051/B/A/IN.CV ISTWERT_MIN|0| |20|0|N|
|1-5-27-11-6 (1)|BP120X1QI9051/B/A/IN.CV ISTWERT_MAX|0| |20|0|N|

AVD-005-MAB_INOCULUM_1

Page 32 of 2834


In [24]:
# lets checkout an example content for a base_node
print(base_nodes[31].get_content())

(MC1B1_003402) Subculture 1


# Setting up Reranker

In [25]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=5,
    model="BAAI/bge-reranker-large",
)

recursive_query_engine = recursive_index.as_query_engine(
    similarity_top_k=5, node_postprocessors=[reranker], verbose=True
)

## For comparison, lets setup a naive RAG pipeline with default parsing and standard chunking, indexing, retrieval.

In [26]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["/Users/angelmurillo/Desktop/OpenSource_RAG_LLM/data/AVD-005-MAB_INOCULUM_1_998pgs.pdf"])
base_docs = reader.load_data()
raw_index = VectorStoreIndex.from_documents(base_docs)
raw_query_engine = raw_index.as_query_engine(
    similarity_top_k=5, node_postprocessors=[reranker]
)

## Testing Queries

In [28]:
query = "What is the Elapsed time from removal of vial from cell bank storage to the start of thawing" #correct answer 61min

response_1 = raw_query_engine.query(query)
print("\n***********Basic Query Engine***********")
print(response_1)

response_2 = recursive_query_engine.query(query)
print("\n***********New LlamaParse+ Recursive Retriever Query Engine***********")
print(response_2)


***********Basic Query Engine***********
61 min
Retrieval entering 3b4fd94d-d7de-434d-8de0-149ead4cd1d0: TextNode
Retrieving from object TextNode with query What is the Elapsed time from removal of vial from cell bank storage to the start of thawing
Retrieval entering 2bb6bbb6-973a-4c04-b8f7-c8cd756bf2fe: TextNode
Retrieving from object TextNode with query What is the Elapsed time from removal of vial from cell bank storage to the start of thawing

***********New LlamaParse+ Recursive Retriever Query Engine***********
The elapsed time from removal of the vial from cell bank storage to the start of thawing is 6 minutes.


In [30]:
query = "What is the cell bank ID? It should be near the scan the label of the ThawSTAR transporter section" #correct answer 1218-W

response_1 = raw_query_engine.query(query)
print("\n***********Basic Query Engine***********")
print(response_1)

response_2 = recursive_query_engine.query(query)
print("\n***********New LlamaParse+ Recursive Retriever Query Engine***********")
print(response_2)


***********Basic Query Engine***********
1234-M
Retrieval entering bec9dccb-55f1-4df0-a925-7df338c9b488: TextNode
Retrieving from object TextNode with query What is the cell bank ID? It should be near the scan the label of the ThawSTAR transporter section
Retrieval entering cd8d2ba0-402c-49b2-a6c7-cd3689a0f4f5: TextNode
Retrieving from object TextNode with query What is the cell bank ID? It should be near the scan the label of the ThawSTAR transporter section
Retrieval entering 40b07b03-078a-426f-ad5d-5ed97dade8fc: TextNode
Retrieving from object TextNode with query What is the cell bank ID? It should be near the scan the label of the ThawSTAR transporter section

***********New LlamaParse+ Recursive Retriever Query Engine***********
1234-M


In [31]:
query = "what is the Elapsed time from removal of vial from cell bank storage to the start of thawing" #correct answer 61min

response_1 = raw_query_engine.query(query)
print("\n***********Basic Query Engine***********")
print(response_1)

response_2 = recursive_query_engine.query(query)
print("\n***********New LlamaParse+ Recursive Retriever Query Engine***********")
print(response_2)


***********Basic Query Engine***********
The Elapsed time from removal of vial from cell bank storage to the start of thawing is 61 minutes.
Retrieval entering 3b4fd94d-d7de-434d-8de0-149ead4cd1d0: TextNode
Retrieving from object TextNode with query what is the Elapsed time from removal of vial from cell bank storage to the start of thawing
Retrieval entering 2bb6bbb6-973a-4c04-b8f7-c8cd756bf2fe: TextNode
Retrieving from object TextNode with query what is the Elapsed time from removal of vial from cell bank storage to the start of thawing

***********New LlamaParse+ Recursive Retriever Query Engine***********
The elapsed time from removal of the vial from cell bank storage to the start of thawing is 6 minutes.


## Using Redis to cache prompts

In [32]:
import redis
import hashlib
import json
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.postprocessor import SentenceTransformerRerank

In [33]:
# Set up Redis connection

redis_client = redis.Redis(
  host='redis-13600.c274.us-east-1-3.ec2.redns.redis-cloud.com',
  port=13600,
  password='k2WbUc23FzUf4x3Kgb3qDJ40NeCAu6Lr')

# Set up RAG components
reader = SimpleDirectoryReader(input_files=["/Users/angelmurillo/Desktop/OpenSource_RAG_LLM/data/AVD-005-MAB_INOCULUM_1_998pgs.pdf"])
base_docs = reader.load_data()
raw_index = VectorStoreIndex.from_documents(base_docs)
reranker = reranker
raw_query_engine = raw_index.as_query_engine(
    similarity_top_k=5, node_postprocessors=[reranker]
)

def get_cache_key(query):
    # Create a unique key based on the query
    return hashlib.md5(query.encode()).hexdigest()

def get_cached_result(query):
    cache_key = get_cache_key(query)
    cached_result = redis_client.get(cache_key)
    if cached_result:
        return json.loads(cached_result)
    return None

def set_cached_result(query, result, expiration_time=3600):
    cache_key = get_cache_key(query)
    redis_client.setex(cache_key, expiration_time, json.dumps(result))

def process_query(query):
    # Check if result is in cache
    cached_result = get_cached_result(query)
    if cached_result:
        print("Cache hit")
        return cached_result

    # If not in cache, perform the query processing
    print("Cache miss. Processing query...")
    result = raw_query_engine.query(query)

    # Convert the result to a serializable format
    serializable_result = {
        'response': str(result.response),
        'source_nodes': [
            {
                'node_id': node.node.node_id,
                'score': node.score,
                'text': node.node.text,
            } for node in result.source_nodes
        ]
    }

    # Cache the result
    set_cached_result(query, serializable_result)

    return serializable_result

In [34]:
# Usage example
if __name__ == "__main__":
    query = "What is the lapsed time from removal of vial from cell bank storage to the start of thawing?"

    # First query - should be a cache miss
    result = process_query(query)
    print(result['response'])

    # Second query with the same parameters - should be a cache hit
    result = process_query(query)
    print(result['response'])

Cache miss. Processing query...
The lapsed time from removal of vial from cell bank storage to the start of thawing is 61 minutes.
Cache hit!
The lapsed time from removal of vial from cell bank storage to the start of thawing is 61 minutes.
